In [1]:
#import libraries

import numpy as np
from scipy.stats import norm
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten, BatchNormalization
from sklearn.metrics import r2_score
import data_ML as ML

np.set_printoptions(suppress=True)

In [2]:
data = []
df = ML.getdata(True)

df_calls = df[df.optionType == 1].drop(columns='optionType')
df_american_calls = df_calls[df_calls.method == 1].drop(columns='method')
df_american_calls
df_american_calls = df_american_calls[df_american_calls.lastPrice > 5]
#training and test datasets
y = np.asarray(df_american_calls[['lastPrice']])
df_american_calls = df_american_calls.drop(columns='lastPrice')
df_american_calls
y


array([[125.7 ],
       [100.21],
       [ 50.77],
       ...,
       [227.33],
       [193.25],
       [199.  ]])

In [3]:
for line in range(len(df_american_calls)):
    data_tmp = df_american_calls.iloc[line]
    data.append(data_tmp)

In [4]:
data = np.asarray(data)
X = data.reshape((data.shape[0], data.shape[1], 1))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
y

array([[125.7 ],
       [100.21],
       [ 50.77],
       ...,
       [227.33],
       [193.25],
       [199.  ]])

In [5]:
#ANN with four layers, 10 neurons each
#activation function: ReLU

# model = Sequential()
# model.add(Dense(128, input_dim=5, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(1))

model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', padding='same', input_shape=(X.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2, strides=3))
model.add(Conv1D(filters=16, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=2, strides=3))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))

# model = Sequential()
# model.add(Conv1D(filters=32, kernel_size=3, activation='relu', padding='same', input_shape=(X.shape[1], 1)))
# model.add(BatchNormalization())
# model.add(MaxPooling1D(pool_size=2, strides=3))
# model.add(Conv1D(filters=16, kernel_size=3, activation='relu', padding='same'))
# model.add(BatchNormalization())
# model.add(MaxPooling1D(pool_size=2, strides=3))
# model.add(Flatten())
# model.add(Dense(64, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))
# model.add(Dense(1))

# from keras.callbacks import EarlyStopping

# # early_stop = EarlyStopping(monitor='val_loss', patience=15, verbose=1)

# from keras.optimizers import SGD, RMSprop

# sgd = SGD(lr=0.01, momentum=0.9)
# rmsprop = RMSprop(lr=0.001)

# model.compile(optimizer=rmsprop, loss='mse')
model.compile(optimizer='adam', loss='mse')

history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)



Epoch 1/100
54/54 [==============================] - 0s 2ms/step - loss: 68521.0625 - val_loss: 52388.3477
Epoch 2/100
54/54 [==============================] - 0s 871us/step - loss: 60931.1133 - val_loss: 50066.0703
Epoch 3/100
54/54 [==============================] - 0s 873us/step - loss: 60013.2227 - val_loss: 48858.5625
Epoch 4/100
54/54 [==============================] - 0s 865us/step - loss: 56240.7461 - val_loss: 47065.3633
Epoch 5/100
54/54 [==============================] - 0s 950us/step - loss: 56699.6328 - val_loss: 46683.0977
Epoch 6/100
54/54 [==============================] - 0s 955us/step - loss: 55788.3945 - val_loss: 46092.4023
Epoch 7/100
54/54 [==============================] - 0s 921us/step - loss: 55213.4727 - val_loss: 45546.2656
Epoch 8/100
54/54 [==============================] - 0s 915us/step - loss: 53629.3945 - val_loss: 44900.6133
Epoch 9/100
54/54 [==============================] - 0s 1ms/step - loss: 53332.4648 - val_loss: 43364.7461
Epoch 10/100
54/54 [===

In [6]:
#prediction
import pandas as pd
y_pred = model.predict(X_test)

db = pd.DataFrame({'prediction':y_pred.tolist(),'real':y_test.tolist()})
db



23/23 [==============================] - 0s 440us/step


,prediction,real
0,[50.415489196777344],[35.97]
1,[269.2870178222656],[258.87]
2,[1239.4378662109375],[1340.65]
3,[126.57205963134766],[107.78]
4,[98.05445861816406],[87.68]
...,...,...
707,[63.60062026977539],[53.5]
708,[64.991943359375],[34.15]
709,[810.6425170898438],[827.83]
710,[152.53683471679688],[154.31]
